In [ ]:
%matplotlib inline
import pandas as pd
import bs4 as bs
import urllib.request
from datetime import datetime
import sqlite3
from finance_data_extract import get_long_historical_price

In [ ]:
df=get_long_historical_price("^GSPC", datetime.strptime('Jan 1 1990', '%b %d %Y').date()
                     , datetime.now().date()
                     , frequency='1d',display=True)

In [ ]:
df.to_csv('sp500.csv')

In [ ]:
df = pd.read_csv('sp500.csv', sep=",",thousands=',')

In [ ]:
df['Date']=pd.to_datetime(df['Date'])

In [ ]:
df = df.set_index('Date')

In [ ]:
df['6 month return']=df['Adj Close'].pct_change(periods=126).shift(-126)
df['1 year return']=df['Adj Close'].pct_change(periods=251).shift(-251)
df['3 year return']=df['Adj Close'].pct_change(periods=753).shift(-753)
df['5 year return'] = df['Adj Close'].pct_change(periods=1255).shift(-1255)
df['10 year return']=df['Adj Close'].pct_change(periods=1510).shift(-1510)
df.head()

In [ ]:
returns_df = df[['6 month return', '1 year return', '3 year return', '5 year return', '10 year return']]

returns_df.head()

In [ ]:
returns_df['6 month return']=returns_df['6 month return'].apply(lambda x: ((1+x)**(1/0.5)-1)*100)
returns_df['1 year return']=returns_df['1 year return'].apply(lambda x: ((1+x)**(1/1)-1)*100)
returns_df['3 year return']=returns_df['3 year return'].apply(lambda x: ((1+x)**(1/3)-1)*100)
returns_df['5 year return']=returns_df['5 year return'].apply(lambda x: ((1+x)**(1/5)-1)*100)
returns_df['10 year return']=returns_df['10 year return'].apply(lambda x: ((1+x)**(1/10)-1)*100)

In [ ]:
conn = sqlite3.connect("treasury_yield.db")
ratedata = pd.read_sql('select * from treasury_yield', conn)

In [ ]:

ratedata['10-2 spread']=ratedata['YR_10']-ratedata['YR_2']


ratedata['Date']=pd.to_datetime(ratedata['Date'], format='%Y-%m-%d')

ratedata.set_index('Date', inplace=True)

ratedata

In [ ]:
merge_df=pd.merge(ratedata, returns_df, left_index=True, right_index=True, how='outer')

merge_df.head(2000)

In [ ]:
plots_df=merge_df[['10-2 spread', '10 year return']]

In [ ]:
plots_df.plot()

In [ ]:
conn = sqlite3.connect('sp500_return.db')
database_df.to_sql(name='rolling_returns', con=conn, if_exists='replace')
conn.close()